In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
import numpy as np

import matplotlib.pyplot as plt
import AutomationModule as AM

In [2]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [3]:
folder_path = "/Users/francisco/Desktop/phd stuff/tidy3d/structures/23_02_2024"
runtime = 120
min_steps_per_lambda = 25
for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction="z", lambda_range=[7.0,4.0],box_size=14.3,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda)
            
            while structure_1.sim.num_time_steps >= 50000:
                  print(structure_1.sim.num_time_steps)
                  runtime -= 10
                  structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction="z", lambda_range=[7.0,4.0],box_size=14.3,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda)
            
            
            while structure_1.sim.num_cells >= 1e8:
                  print(structure_1.sim.num_cells)
                  min_steps_per_lambda -= 2
                  structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction="z", lambda_range=[7.0,4.0],box_size=14.3,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda)
            
            
            min_steps_per_lambda = 22
            runtime = 120   
                  

            results = structure_1.run_sim(folder_description="2023-02-2024_Free_Runs",load=False)


            #sim_data, sim_data0, task_name_def = results

            #Plot flux (improve plots and make everything cleaner pending.)
             # Retrieve the power flux through the monitor plane.
            #transmission0 = sim_data0['flux1'].flux
            #transmission = sim_data['flux1'].flux
            #transmission_normalized = transmission / transmission0
            #transmission_flux_right = sim_data['flux1'].flux
            #transmission_flux_left = sim_data['flux2'].flux
            #monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
            #plt.semilogy(monitor_lambdas, (transmission_flux_left/transmission0), label='Left Monitor')
            #plt.semilogy(monitor_lambdas, (transmission_flux_right/transmission0), label='Right Monitor')
            #plt.xlabel('$\lambda (\mu m)$')
            #plt.ylabel('Transmission')
            #plt.legend()
            #plt.ylim(1e-2,1)
            #plt.grid()
            #plt.title(f"{task_name_def}\n")
            #plt.show()
    #

            


Configured successfully.
60294
Configured successfully.
55269
Configured successfully.
50245
Configured successfully.
Configured successfully.
53100
Configured successfully.
Configured successfully.
53100
Configured successfully.
